In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import random

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import roc_auc_score
from sentence_transformers import SentenceTransformer
import matplotlib.pyplot as plt
from tqdm import tqdm

Load all data

In [ ]:
def load_news(news_path):
    df = pd.read_csv(news_path, sep='\t', header=None,
                     names=['news_id', 'category', 'subcategory', 'title', 'abstract', 'url', 'title_entities', 'abstract_entities'])
    df['content'] = df['title'].fillna('') + ' ' + df['abstract'].fillna('')
    return df[['news_id', 'content']]

train_news = load_news('MINDsmall_train/news.tsv')
dev_news = load_news('MINDsmall_dev/news.tsv')
# Check for duplicates on news_id and content
all_news = pd.concat([train_news, dev_news]).drop_duplicates(['news_id', 'content'])

In [ ]:
all_news.head()

In [ ]:
all_news.loc[0].content

In [ ]:
all_news.describe()

In [ ]:
print("Size of the train dataset:", train_news.shape[0])
print("Size of the test dataset:", dev_news.shape[0])

train TF-IDF

In [ ]:
# TF-IDF vectorization
vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
news_vectors = vectorizer.fit_transform(train_news['content'])  # only train content for training

# Map news_id to index
news_id_to_index = dict(zip(train_news['news_id'], range(len(train_news))))

Load Training Behaviors and Build User Profiles

In [ ]:
train_behaviors = pd.read_csv('MINDsmall_train/behaviors.tsv', sep='\t', header=None,
                              names=['impression_id', 'user_id', 'time', 'history', 'impressions'])

user_profiles = {}

for _, row in train_behaviors.iterrows():
    user = row['user_id']
    history = row['history']
    if pd.isna(history): continue
    history_ids = [nid for nid in history.split() if nid in news_id_to_index]
    if not history_ids: continue
    indices = [news_id_to_index[nid] for nid in history_ids]
    profile_vector = news_vectors[indices].mean(axis=0)
    user_profiles[user] = profile_vector

Calculate embeddings

In [ ]:
# Load a pre-trained SentenceTransformer model for generating sentence embeddings
# 'all-MiniLM-L6-v2' (384-dimensional vector space) is a lightweight, efficient model good for semantic similarity tasks
model = SentenceTransformer('all-MiniLM-L6-v2')

# Compute embeddings for all news article content in the training set
embeddings = model.encode(train_news['content'].tolist(), show_progress_bar=True)

Build user profiles using the embeddings

In [ ]:
user_embed_profiles = {}

for _, row in train_behaviors.iterrows():
    user = row['user_id']
    history = row['history']
    if pd.isna(history): continue
    history_ids = [nid for nid in history.split() if nid in news_id_to_index]
    if not history_ids: continue
    indices = [news_id_to_index[nid] for nid in history_ids]
    # Compute the mean embedding vector for the user's history
    user_embedding = np.mean([embeddings[i] for i in indices], axis=0)
    # Store the reshaped embedding (as a 2D array) in the dictionary, to be used later for cosine similarity
    user_embed_profiles[user] = user_embedding.reshape(1, -1)

Evaluate the model on the dev set

These four metrics—**AUC**, **MRR**, **nDCG\@5**, and **nDCG\@10**—are commonly used to evaluate the performance of recommendation or ranking systems.
1. **AUC (Area Under the Curve)** measures the ability of the model to distinguish between positive and negative instances, with values closer to 1 indicating better discrimination.
2. **MRR (Mean Reciprocal Rank)** evaluates how highly the first relevant item appears in the ranked list, rewarding models that place relevant items near the top.
3. **nDCG\@5** and **nDCG\@10** (Normalized Discounted Cumulative Gain at positions 5 and 10) assess the quality of the top-ranked results by giving higher importance to relevant items appearing earlier in the list, normalized to account for the best possible ranking.


In [ ]:
dev_behaviors = pd.read_csv('MINDsmall_dev/behaviors.tsv', sep='\t', header=None,
                            names=['impression_id', 'user_id', 'time', 'history', 'impressions'])


def mrr(scores, labels):
    # Combine scores and labels, sort them in descending order of score
    sorted_labels = [l for _, l in sorted(zip(scores, labels), reverse=True)]
    for i, label in enumerate(sorted_labels):
        if label == 1:
            return 1.0 / (i + 1)
    return 0.0

def ndcg(scores, labels, k):
    sorted_labels = [l for _, l in sorted(zip(scores, labels), reverse=True)][:k]
    dcg = sum([1.0 / np.log2(i + 2) if l == 1 else 0 for i, l in enumerate(sorted_labels)])
    ideal_dcg = sum([1.0 / np.log2(i + 2) for i in range(min(sum(labels), k))])
    return dcg / ideal_dcg if ideal_dcg > 0 else 0.0

def evaluate(dev_behaviors, news_id_to_index, news_vectors, user_profiles):
    aucs, mrrs, ndcg5s, ndcg10s = [], [], [], []

    for _, row in tqdm(dev_behaviors.iterrows(), total=len(dev_behaviors)):
        user = row['user_id']
        impressions = [imp.split('-') for imp in row['impressions'].split()]
        news_ids = [nid for nid, _ in impressions if nid in news_id_to_index]
        labels = [int(label) for nid, label in impressions if nid in news_id_to_index]

        if not news_ids or user not in user_profiles or len(set(labels)) == 1:
            continue

        profile_vector = np.asarray(user_profiles[user])  # Convert to numpy array
        candidate_idxs = [news_id_to_index[nid] for nid in news_ids]
        candidate_vectors = news_vectors[candidate_idxs]
        # Compute cosine similarity between user profile and each news candidate
        sims = cosine_similarity(profile_vector, candidate_vectors).flatten()

        aucs.append(roc_auc_score(labels, sims, multi_class='ovr'))
        mrrs.append(mrr(sims, labels))
        ndcg5s.append(ndcg(sims, labels, 5))
        ndcg10s.append(ndcg(sims, labels, 10))

    return {
        'AUC': np.mean(aucs),
        'MRR': np.mean(mrrs),
        'nDCG@5': np.mean(ndcg5s),
        'nDCG@10': np.mean(ndcg10s)
    }

Evaluate a baseline system (random reccomender)

In [ ]:
def evaluate_random(dev_behaviors, news_id_to_index):
    aucs, mrrs, ndcg5s, ndcg10s = [], [], [], []

    for _, row in tqdm(dev_behaviors.iterrows(), total=len(dev_behaviors)):
        impressions = [imp.split('-') for imp in row['impressions'].split()]
        news_ids = [nid for nid, _ in impressions if nid in news_id_to_index]
        labels = [int(label) for nid, label in impressions if nid in news_id_to_index]

        if not news_ids or len(set(labels)) == 1:
            continue

        random_scores = [random.random() for _ in labels]

        aucs.append(roc_auc_score(labels, random_scores))
        mrrs.append(mrr(random_scores, labels))
        ndcg5s.append(ndcg(random_scores, labels, 5))
        ndcg10s.append(ndcg(random_scores, labels, 10))

    return {
        'AUC': np.mean(aucs),
        'MRR': np.mean(mrrs),
        'nDCG@5': np.mean(ndcg5s),
        'nDCG@10': np.mean(ndcg10s)
    }

Compare results

In [ ]:
cb_results = evaluate(dev_behaviors, news_id_to_index, news_vectors, user_profiles)
rand_results = evaluate_random(dev_behaviors, news_id_to_index)
embed_results = evaluate(dev_behaviors, news_id_to_index, embeddings, user_embed_profiles)

# Display comparison
print("Content-Based Recommender Performance:")
for metric, score in cb_results.items():
    print(f"{metric}: {score:.4f}")

print("\nContent-Based Embedding Recommender Performance:")
for metric, score in embed_results.items():
    print(f"{metric}: {score:.4f}")

print("\nRandom Recommender Performance:")
for metric, score in rand_results.items():
    print(f"{metric}: {score:.4f}")

In [ ]:
# Combine results into a single dictionary for easy plotting
all_results = {
    "Random": rand_results,
    "Content-Based": cb_results,
    "Embedding-Based": embed_results    
}

# Round the results to 4 decimal places
for model, metrics in all_results.items():
    for metric, score in metrics.items():
        all_results[model][metric] = round(score, 4)

# Convert the results into a DataFrame for easier plotting
results_df = pd.DataFrame(all_results).T
results_df.reset_index(inplace=True)
results_df.columns = ['Model', 'AUC', 'MRR', 'nDCG@5', 'nDCG@10']
results_df = results_df.melt(id_vars='Model', var_name='Metric', value_name='Score')
# Set the figure size
plt.figure(figsize=(12, 6))
# Create a bar plot
ax = sns.barplot(x='Metric', y='Score', hue='Model', data=results_df)
ax.bar_label(ax.containers[0])
ax.bar_label(ax.containers[1])
ax.bar_label(ax.containers[2])
# Set the title and labels
plt.title('Model Performance Comparison')
plt.xlabel('Metrics')
plt.ylabel('Score')
# Show the plot
plt.legend(title='Model')
plt.show()